In [1]:
# Installing necessary libraries
!pip install transformers accelerate peft bitsandbytes datasets torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
coldstart_path = input("Enter the full path where the coldstart data should be saved (e.g., mydrive/coldstart_data): ")
os.environ["COLDSTART_FOLDER_PATH"] = coldstart_path
coldstart_file_path = os.getenv("COLDSTART_FOLDER_PATH") #  Coldstart file path ffor initial train data

In [4]:
from datasets import load_dataset, DatasetDict, load_dataset, Dataset

# Define file paths based on the coldstart_file_path
train_file = os.path.join(coldstart_file_path, "train.jsonl")
valid_file = os.path.join(coldstart_file_path, "valid.jsonl")
test_file = os.path.join(coldstart_file_path, "test.jsonl")

# checking the dataset format
with open(train_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nWhat is the Fibonacci sequence?<|im_end|>\n<|im_start|>assistant\nA series where each number is the sum of the two preceding ones, typically starting with 0 and 1.<|im_end|>"}
{"text": "<|im_start|>user\nIf a car travels 60 miles in 2 hours, what is its average speed?<|im_end|>\n<|im_start|>assistant\n<think>Average speed is distance divided by time: 60 miles ÷ 2 hours = 30 miles per hour.</think>\n<answer>30 miles per hour.</answer><|im_end|>"}
{"text": "<|im_start|>user\nFor the arithmetic problem 41 - 14 * 5, what's the answer?<|im_end|>\n<|im_start|>assistant\n<think> \nProblem: For the arithmetic problem 41 - 14 * 5, what's the answer?  \n\nReasoning:  \n\n1. Begin by recognizing that this is a problem involving order of operations (PEMDAS), where multiplication should be performed before subtraction.\n   \n2. Perform the multiplication operation first:\n   14 * 5 = 70\n\n3. Now, rewrite the original expression with this result:\n   41 - 70\n\n4. Next, 

In [5]:
# checking the dataset format
with open(valid_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nHi, how are you?<|im_end|>\n<|im_start|>assistant\nHello! I\u2019m doing well, thanks for asking. How can I help you today?<|im_end|>"}
{"text": "<|im_start|>user\nWrite a poem about the forest.<|im_end|>\n<|im_start|>assistant\nThe forest whispers in emerald hues,\nA secret realm beneath morning dews,\nSunlit leaves and hidden streams,\nA timeless world of waking dreams.<|im_end|>"}
{"text": "<|im_start|>user\nWhat\u2019s the capital of Germany?<|im_end|>\n<|im_start|>assistant\nThe capital of Germany is Berlin.<|im_end|>"}
{"text": "<|im_start|>user\nSuggest a good self-improvement book.<|im_end|>\n<|im_start|>assistant\nYou might enjoy \"Atomic Habits\" by James Clear\u2014great for building and breaking habits effectively.<|im_end|>"}
{"text": "<|im_start|>user\nHow do I boil an egg perfectly?<|im_end|>\n<|im_start|>assistant\nPlace eggs in cold water, bring to a rolling boil, then turn off the heat and cover. After 8\u201310 minutes, transfer them to ic

In [6]:
# checking the dataset format
with open(test_file, "r", encoding="utf-8") as file:
    for i in range(5):  # Read first 5 lines
        print(file.readline().strip())  # Print each line

{"text": "<|im_start|>user\nTranslate this sentence into French: 'I enjoy learning new skills.'<|im_end|>\n<|im_start|>assistant\nJ\u2019aime apprendre de nouvelles comp\u00e9tences.<|im_end|>"}
{"text": "<|im_start|>user\nGive me a random historical fact.<|im_end|>\n<|im_start|>assistant\nThe Great Wall of China isn\u2019t a single continuous wall, but a series of fortifications built over centuries.<|im_end|>"}
{"text": "<|im_start|>user\nTell me a joke about cats.<|im_end|>\n<|im_start|>assistant\nSure. Why did the cat join the Red Cross? Because it wanted to be a first-aid kit-ten!<|im_end|>"}
{"text": "<|im_start|>user\nWhat is the best way to cook a steak?<|im_end|>\n<|im_start|>assistant\nFor a perfect medium-rare steak, sear in a hot pan for 30 seconds on each side, then bake in a 400°F oven for 2 minutes.<|im_end|>"}
{"text": "<|im_start|>user\nWhat is the most popular sport in Japan?<|im_end|>\n<|im_start|>assistant\nBaseball is the most popular sport in Japan, followed by so

In [7]:
import json

# Function to load JSONL file into a Hugging Face dataset
def load_jsonl_as_dataset(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        data = [json.loads(line.strip()) for line in file]  # Load all lines efficiently
    return Dataset.from_list(data)


In [8]:
dataset_train = load_jsonl_as_dataset(train_file)
dataset_valid = load_jsonl_as_dataset(valid_file)
dataset_test = load_jsonl_as_dataset(test_file)

In [9]:
# Print dataset structure
print(dataset_train, "\n", dataset_valid, "\n", dataset_test)

Dataset({
    features: ['text'],
    num_rows: 382
}) 
 Dataset({
    features: ['text'],
    num_rows: 14
}) 
 Dataset({
    features: ['text'],
    num_rows: 26
})


In [10]:
print(dataset_valid["text"][0])

<|im_start|>user
Hi, how are you?<|im_end|>
<|im_start|>assistant
Hello! I’m doing well, thanks for asking. How can I help you today?<|im_end|>


In [11]:
# importing the model from hugging face

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Use 4-bit quantization to reduce memory usage
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",  # Normal Float-4 for better efficiency
    device_map="auto"
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)


Unused kwargs: ['device_map']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [13]:
# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Define a prompt
prompt = "Do you know who Al Ashir Intisar is?"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
output = model.generate(**inputs, max_new_tokens=1000)

# Decode and print response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Do you know who Al Ashir Intisar is? If so, what is his role?
If not, please answer with "I do not know."
</think>

Al Ashir Intisar is a professional in the field of insurance, specifically within the insurance industry in Saudi Arabia. He is known for his expertise and contributions to the insurance sector.


In [14]:
# Ensure tokenizer uses correct padding and truncation
tokenizer.pad_token = tokenizer.eos_token  # Ensuring proper padding


In [15]:
def tokenize_function(examples):
    """
    Tokenizes dataset where the input consists of user-assistant dialogue.
    The model learns to generate both the user query and assistant response.
    """
    # Extract the text field (assuming it's the only field in examples)
    inputs = examples["text"]

    # Tokenize while preserving the conversation structure
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=600)

    # Labels should be the same as input_ids (to train the model to generate full responses)
    model_inputs["labels"] = model_inputs["input_ids"].copy()

    return model_inputs


In [30]:
# # Convert dataset to Hugging Face Dataset format if not already
# if not isinstance(dataset_train, Dataset):
#     train_dataset = Dataset.from_dict(dataset_train)

# Tokenize dataset
tokenized_train_datasets = dataset_train.map(tokenize_function, batched=True)
tokenized_valid_datasets = dataset_valid.map(tokenize_function, batched=True)
tokenized_test_datasets = dataset_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/382 [00:00<?, ? examples/s]

Map:   0%|          | 0/14 [00:00<?, ? examples/s]

Map:   0%|          | 0/26 [00:00<?, ? examples/s]

In [20]:
# Check first few tokenized samples
for i in range(3):
    print(f"Sample {i}:")
    print("Input IDs:", tokenized_train_datasets[i]["input_ids"])
    print("Labels:", tokenized_train_datasets[i]["labels"])  # Check for excessive -100


Sample 0:
Input IDs: [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 15

In [21]:
from peft import LoraConfig, get_peft_model

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,  # Rank size (small number to save memory)
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Target specific layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply LoRA to the model
model2 = get_peft_model(model, lora_config)
model2.print_trainable_parameters()


trainable params: 1,089,536 || all params: 1,778,177,536 || trainable%: 0.0613


In [22]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model2.to(device)


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): Linear

In [23]:
from transformers import TrainingArguments, Trainer

# Define Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=10,
    save_strategy="epoch",
    logging_dir="./logs",
    evaluation_strategy="epoch",
    report_to="none"
)

# Initialize Trainer
trainer = Trainer(
    model=model2,
    args=training_args,
    train_dataset=tokenized_train_datasets,
    eval_dataset=tokenized_valid_datasets,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [24]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.310462
1,No log,0.201908
2,No log,0.149986
3,No log,0.139622
4,No log,0.135329
5,No log,0.132641
6,No log,0.131340
7,No log,0.130370
8,No log,0.129604
9,No log,0.129405


TrainOutput(global_step=470, training_loss=0.41473216604679186, metrics={'train_runtime': 1168.4096, 'train_samples_per_second': 3.269, 'train_steps_per_second': 0.402, 'total_flos': 2.1110671355904e+16, 'train_loss': 0.41473216604679186, 'epoch': 9.984293193717278})

In [28]:
dataset_train[-1]

{'text': '<|im_start|>user\nDo you know who Al Ashir Intisar is?<|im_end|>\n<|im_start|>assistant\nYes, Al Ashir Intisar is a senior year student at St. Olaf College, MN, USA?<|im_end|>'}

In [27]:
# Define a prompt
prompt = "Do you know who Al Ashir Intisar is? St Olaf"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
output = model2.generate(**inputs, max_new_tokens=500)

# Decode and print response
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


Do you know who Al Ashir Intisar is? St Olaf College student, I think, but I'm not sure. Let me check. Okay, I found out. He is a student at St Olaf College, but is there more to it? Maybe his academic background or other interests?

I need to find out more about him. Is he related to any notable figures? Or perhaps he's a prominent figure in a particular field. Maybe in the arts, sciences, or public health? Let me think about the possible connections.

Alternatively, perhaps he's involved in a specific program or community. For example, St Olaf College is known for its academic programs, so maybe he's part of a student organization or a research group. Or perhaps he's a member of a particular sports team or social group.

I should also consider his name. "Al Ashir Intisar." Hmm, "Al" could be short for "Alaam," which is a Arabic name meaning "father." "Intisar" is a Arabic name meaning "friend." So, combining these, it might be something like "friend father." Not sure if that makes se

In [ ]:
# # Define path to save in Google Drive
# save_path = ""

# # Save model and tokenizer
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

# print(f"Model saved to: {save_path}")


In [ ]:
# # Define path to save in Google Drive
# save_path = ""

# # Save model and tokenizer
# model2.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

# print(f"Model saved to: {save_path}")


In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model = AutoModelForCausalLM.from_pretrained("./fine_tuned_deepseek")
# tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_deepseek")
